##Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
print(tf.__version__)

##Data Augmentation

In [ ]:
#Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.3,
                                   horizontal_flip=True,
                                   rotation_range=50,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   brightness_range=[0.8, 1.2], #Add brightness variation
                                   fill_mode='nearest')

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_set = train_datagen.flow_from_directory('dataset/test_set',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

##Load the pre-trained VGG16 model


In [ ]:
vgg16_base = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(64, 64, 3)) #load the pre-trained VGG16 model without the top fully connected layers

##Freeze the base model layers

In [ ]:
vgg16_base.trainable = False #Freeze the base model layers so they are not updated during training

##Define the Sequential and VGG16 models

In [ ]:
#Define the Sequential model and add the VGG16 base model
model = Sequential([
    vgg16_base,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),  # Adjusted dropout to 40% for regularization
    Dense(1, activation='sigmoid')
])

##Compiling

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
              loss='binary_crossentropy',
              metrics=['accuracy']) #Compiling the model

##Implement Early Stopping

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',   # Monitor validation loss
    patience=3,           # Stop training after 3 epochs of no improvement
    restore_best_weights=True  # Restore the best weights once training stops
)

##Import class weights to avoid imbalances

In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Get labels from the training set
labels = training_set.classes

# Compute class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights = dict(enumerate(class_weights))

##Training the model

In [ ]:
model.fit(
    training_set,
    validation_data=test_set,
    epochs=30,
    callbacks=[early_stopping],
    class_weight=class_weights #apply class wights if there is an imbalance
)

##Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/cat_dog_8.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)  # Expand to match the batch size for the model
test_image = test_image / 255.0  # Normalize the image

In [ ]:
result = model.predict(test_image) #Make the prediction using the trained model

##Output prediction result

In [ ]:
if result [0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'

In [ ]:
print(prediction)